## Spark SQL 튜토리얼
+ 데이터프레임 생성하기
+ SQL 처리를 위한 Temporary View 생성
+ 기본 SQL문 실습

In [1]:
# [+] SparkSession 임포트
from pyspark.sql import SparkSession

In [2]:
# [+] SparkSession 설정하기
ss = SparkSession.builder.master('local').appName('learn-sql').getOrCreate()

### 샘플 데이터셋: 주식 데이터
+ 변수: 회사명(name), 종목코드(ticker), 상장국가(country), 주가(price), 통화(currency)
+ 주가: Double 형식과 Long 형식이 섞이면 TypeError 발생하므로 Double(실수) 형태로 통일
    + 예: 삼성 주가, 67300 -> 67300.0


In [3]:
# 샘플 데이터셋
"""
주가는 int형 변수이기 때문에 에러가 발생한다. 그래서 .0(소수점)으로 표현하여
double형으로 사용하겠다는것을 알려준다
숫자를 그냥 작성시 String형으로 받는다.
"""
stocks = [
    ('Alphabet', 'GOOGL', 'USA', 2282.19, 'USD'), 
    ('Netflix', 'NFLX', 'USA', 190.36, 'USD'),
    ('Amazon', 'AMZN', 'USA', 2485.63, 'USD'),
    ('Tesla', 'TSLA', 'USA', 870.76, 'USD'),
    ('HUYA', 'HUYA', 'USA', 4.13, 'USD'),
    ('Tencent', '0700', 'Hong Kong', 377.4, 'HKD'),
    ('Toyota', '7203', 'Japan', 2228.0, 'JPY'),
    ('Samsung', '005930', 'Korea', 67300.0, 'KRW'),
    ('Kakao', '035720', 'Korea', 87300.0, 'KRW'),
    ('Com2uS', '078340', 'Korea', 95000.0, 'KRW')
]

In [4]:
# [+] 스키마 정의: 데이터 타입은 자동으로 추론
stocksSchema = ['name', 'ticker', 'country', 'price', 'currency']

In [5]:
# [+] 데이터프레임 생성 
"""
data, schema 매개변수
createDataFrame() 메소드
"""
df = ss.createDataFrame(data=stocks, schema=stocksSchema)

In [6]:
# [+] 생성된 데이터프레임 확인
"""
printSchema() 메소드 

schema를 볼수 있음 데이터 type을 자동으로 추측을 해서 setting을 해준다
"""
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- ticker: string (nullable = true)
 |-- country: string (nullable = true)
 |-- price: double (nullable = true)
 |-- currency: string (nullable = true)



In [7]:
# [+] 데이터프레임 출력

df.show()

+--------+------+---------+-------+--------+
|    name|ticker|  country|  price|currency|
+--------+------+---------+-------+--------+
|Alphabet| GOOGL|      USA|2282.19|     USD|
| Netflix|  NFLX|      USA| 190.36|     USD|
|  Amazon|  AMZN|      USA|2485.63|     USD|
|   Tesla|  TSLA|      USA| 870.76|     USD|
|    HUYA|  HUYA|      USA|   4.13|     USD|
| Tencent|  0700|Hong Kong|  377.4|     HKD|
|  Toyota|  7203|    Japan| 2228.0|     JPY|
| Samsung|005930|    Korea|67300.0|     KRW|
|   Kakao|035720|    Korea|87300.0|     KRW|
|  Com2uS|078340|    Korea|95000.0|     KRW|
+--------+------+---------+-------+--------+



In [8]:
# [+] Temporary View 생성
df.createOrReplaceTempView('stocks')

In [9]:
# [+] SELECT 문: 하나의 컬럼을 가져오기
"""
sql keword는 대문자로 사용한다.
name별 컬럼 값만 가져온다

SELECT * 을 하면 값을 다 가져온다
"""
ss.sql("SELECT name FROM stocks").show()

+--------+
|    name|
+--------+
|Alphabet|
| Netflix|
|  Amazon|
|   Tesla|
|    HUYA|
| Tencent|
|  Toyota|
| Samsung|
|   Kakao|
|  Com2uS|
+--------+



In [11]:
# [+] SELECT 문: 여러 개의 컬럼을 가져오기
ss.sql("SELECT name, price FROM stocks").show()

+--------+-------+
|    name|  price|
+--------+-------+
|Alphabet|2282.19|
| Netflix| 190.36|
|  Amazon|2485.63|
|   Tesla| 870.76|
|    HUYA|   4.13|
| Tencent|  377.4|
|  Toyota| 2228.0|
| Samsung|67300.0|
|   Kakao|87300.0|
|  Com2uS|95000.0|
+--------+-------+



In [13]:
# SELECT 문: WHERE 조건을 만족하는 데이터 선택하기
# [+] 한국에 상장된 주식
"""
Koera 는 ''로 묶어줘야 에러가 발생하지 않는다.

한국에 상장된 주식만 가져온다
"""
ss.sql("SELECT name, country FROM stocks WHERE country='Korea'").show()

+-------+-------+
|   name|country|
+-------+-------+
|Samsung|  Korea|
|  Kakao|  Korea|
| Com2uS|  Korea|
+-------+-------+



In [14]:
# [+] 주가 2000 이상의 주식

ss.sql("SELECT name, price FROM stocks WHERE price >= 2000").show()

+--------+-------+
|    name|  price|
+--------+-------+
|Alphabet|2282.19|
|  Amazon|2485.63|
|  Toyota| 2228.0|
| Samsung|67300.0|
|   Kakao|87300.0|
|  Com2uS|95000.0|
+--------+-------+



In [16]:
# [+] 주가가 2000 이상인 미국 주식

xxxxxxxxxxxx

ss.sql("SELECT name, price FROM stocks WHERE price >=2000 AND currency='USD'").show()

+--------+-------+
|    name|  price|
+--------+-------+
|Alphabet|2282.19|
|  Amazon|2485.63|
+--------+-------+



In [17]:
# LIKE: 문자열 관련 처리할 때 유용
# [+] 상장국가명이 'U'로 시작하는 주식
"""
%n n으로 끝나는 문자열

U% U로 시작하는 물자열

%o% 전제 문자열중에 o가 들어가는 문자열
"""
ss.sql("SELECT name, country FROM stocks WHERE country LIKE 'U%'").show()

+--------+-------+
|    name|country|
+--------+-------+
|Alphabet|    USA|
| Netflix|    USA|
|  Amazon|    USA|
|   Tesla|    USA|
|    HUYA|    USA|
+--------+-------+



In [18]:
# NOT LIKE
# [+] 상장국가명이 'U'로 시작하며, 회사명에 'e' 가 들어가지 않는 주식 

"""
코드의 가독성을 위하여 \(enter)를 사용해 코드 줄을 넘어간다.
"""
ss.sql("SELECT name, country FROM stocks \
        WHERE country LIKE 'U%' AND \
        name NOT LIKE '%e%'").show()

+------+-------+
|  name|country|
+------+-------+
|Amazon|    USA|
|  HUYA|    USA|
+------+-------+



In [19]:
# BETWEEN
# [+] 주가가 1000~10000 사이인 주식
ss.sql("SELECT name, price FROM stocks \
        WHERE price BETWEEN 1000 AND 10000").show()

+--------+-------+
|    name|  price|
+--------+-------+
|Alphabet|2282.19|
|  Amazon|2485.63|
|  Toyota| 2228.0|
+--------+-------+



In [20]:
# 중첩된 SQL 문
# [+] 미국의 주식 중 Tesla 보다 주가가 높은 주식

ss.sql("SELECT name, price, country FROM stocks \
        WHERE country = 'USA' AND \
        price > (SELECT price FROM stocks WHERE ticker = 'TSLA')").show()

+--------+-------+-------+
|    name|  price|country|
+--------+-------+-------+
|Alphabet|2282.19|    USA|
|  Amazon|2485.63|    USA|
+--------+-------+-------+



In [21]:
# ORDER BY
# [+] 주가를 기준으로 오름차순(ASC: Ascending) 정렬
"""
ORDER BY keword
"""
ss.sql("SELECT name, price FROM stocks ORDER BY price ASC").show()

+--------+-------+
|    name|  price|
+--------+-------+
|    HUYA|   4.13|
| Netflix| 190.36|
| Tencent|  377.4|
|   Tesla| 870.76|
|  Toyota| 2228.0|
|Alphabet|2282.19|
|  Amazon|2485.63|
| Samsung|67300.0|
|   Kakao|87300.0|
|  Com2uS|95000.0|
+--------+-------+



In [22]:
# [+] 내림차순(DESC: Descending) 정렬
ss.sql("SELECT name, price FROM stocks ORDER BY price DESC").show()

+--------+-------+
|    name|  price|
+--------+-------+
|  Com2uS|95000.0|
|   Kakao|87300.0|
| Samsung|67300.0|
|  Amazon|2485.63|
|Alphabet|2282.19|
|  Toyota| 2228.0|
|   Tesla| 870.76|
| Tencent|  377.4|
| Netflix| 190.36|
|    HUYA|   4.13|
+--------+-------+



In [23]:
# 집계연산
# [+] 한국 주식의 주가 총합
ss.sql("SELECT sum(price) FROM stocks WHERE country = 'Korea'").show()

+----------+
|sum(price)|
+----------+
|  249600.0|
+----------+



In [24]:
# [+] 한국 주식의 주가 평균
ss.sql("SELECT mean(price) FROM stocks WHERE country = 'Korea'").show()

+-----------+
|mean(price)|
+-----------+
|    83200.0|
+-----------+



In [26]:
# [+] 한국 주식의 개수
ss.sql("SELECT count(name) FROM stocks WHERE country = 'Korea'").show()

+-----------+
|count(name)|
+-----------+
|          3|
+-----------+



In [27]:
# [+] IN 통화가 KRW 또는 JPY 인 주식
ss.sql("SELECT name, currency FROM stocks \
        WHERE currency IN ('KRW', 'JPY')").show()

+-------+--------+
|   name|currency|
+-------+--------+
| Toyota|     JPY|
|Samsung|     KRW|
|  Kakao|     KRW|
| Com2uS|     KRW|
+-------+--------+



### 두 개의 데이터프레임을 이용한 SQL 문
데이터베이스나 빅데이터 환경에서는 여러 개의 데이터 테이블들을 결합하여 query를 수행하는 경우가 빈번함
SQL 문의 Join을 이용하여 stocks 와 earnings 데이터프레임을 결합하여 데이터 분석하기

In [28]:
# earnings = EPS(earning per share): 주당 순이익, 전체 순이익을 전체 주식 수로 나눈 값

earnings = [
    ('Alphabet', 110.57, 'USD'), 
    ('Netflix', 10.46, 'USD'),
    ('Amazon', 41.36, 'USD'),
    ('Tesla', 7.4, 'USD'),
    ('HUYA', 0.36, 'USD'),
    ('Tencent', 27.6, 'HKD'),
    ('Toyota', 221.35, 'JPY'),
    ('Samsung', 5777.37, 'KRW'),
    ('Kakao', 3150.95, 'KRW'),
    ('Com2uS', 10852.69, 'KRW')
]

In [29]:
# [+] 스키마 정의에 필요한 클래스 임포트하기
from pyspark.sql.types import StructType, StructField, StringType, FloatType

In [32]:
# [+] 스키마 정의: 직접 데이터 타입 정하기
"""
True null값을 허용하겠다.
"""
earningSchema = StructType([
    StructField('name', StringType(), True),
    StructField('eps', FloatType(), True),
    StructField('currency', StringType(), True)
])

In [34]:
# [+] 데이터프레임 생성
df_earning = ss.createDataFrame(data=earnings, schema=earningSchema)

In [35]:
# [+] earningDF의 Temporary View 생성
df_earning.createOrReplaceTempView('earnings')

In [38]:
# [+] 함수 형태로 query 수행하기, 예: select()
df_earning.select('*').show()

+--------+--------+--------+
|    name|     eps|currency|
+--------+--------+--------+
|Alphabet|  110.57|     USD|
| Netflix|   10.46|     USD|
|  Amazon|   41.36|     USD|
|   Tesla|     7.4|     USD|
|    HUYA|    0.36|     USD|
| Tencent|    27.6|     HKD|
|  Toyota|  221.35|     JPY|
| Samsung| 5777.37|     KRW|
|   Kakao| 3150.95|     KRW|
|  Com2uS|10852.69|     KRW|
+--------+--------+--------+



In [37]:
# [+] Inner join 수행
ss.sql("SELECT * FROM stocks JOIN earnings ON stocks.name = earnings.name").show()

+--------+------+---------+-------+--------+--------+--------+--------+
|    name|ticker|  country|  price|currency|    name|     eps|currency|
+--------+------+---------+-------+--------+--------+--------+--------+
|Alphabet| GOOGL|      USA|2282.19|     USD|Alphabet|  110.57|     USD|
|  Amazon|  AMZN|      USA|2485.63|     USD|  Amazon|   41.36|     USD|
|  Com2uS|078340|    Korea|95000.0|     KRW|  Com2uS|10852.69|     KRW|
|    HUYA|  HUYA|      USA|   4.13|     USD|    HUYA|    0.36|     USD|
|   Kakao|035720|    Korea|87300.0|     KRW|   Kakao| 3150.95|     KRW|
| Netflix|  NFLX|      USA| 190.36|     USD| Netflix|   10.46|     USD|
| Samsung|005930|    Korea|67300.0|     KRW| Samsung| 5777.37|     KRW|
| Tencent|  0700|Hong Kong|  377.4|     HKD| Tencent|    27.6|     HKD|
|   Tesla|  TSLA|      USA| 870.76|     USD|   Tesla|     7.4|     USD|
|  Toyota|  7203|    Japan| 2228.0|     JPY|  Toyota|  221.35|     JPY|
+--------+------+---------+-------+--------+--------+--------+--

In [41]:
# PER (Price/Earnings Ratio): 주가를 EPS로 나눈 값
# [+] PER 기준으로 가장 저평가된 주식 찾기
ss.sql("SELECT stocks.name, (stocks.price / earnings.eps) \
        FROM stocks JOIN earnings ON stocks.name = earnings.name \
        ORDER BY (stocks.price / earnings.eps) ASC").show()

+--------+------------------+
|    name|     (price / eps)|
+--------+------------------+
|  Com2uS|  8.75358977716049|
|  Toyota|10.065506837881282|
|    HUYA|11.472221766357087|
| Samsung|11.648898830245365|
| Tencent|13.673912854486094|
| Netflix| 18.19885270609645|
|Alphabet| 20.64022796688883|
|   Kakao|27.705930386971747|
|  Amazon|60.097436250469926|
|   Tesla|117.67026875379577|
+--------+------------------+

